In [ ]:
import json
import pandas as pd
import re

### pull data

In [ ]:
data = {}
with open('intermediate_outputs/raw_formatted_appState.json', 'r') as f:
    data = json.load(f)

### a quick_search test

In [ ]:
quick_search = {
    key: {
        "template_string": str(value.get("template")),
        "type": value["type"]
    }
    for key, value in data.items()
    if str(value.get("template")).strip() != ""
}

In [ ]:
query = "distance_to_dealerships"
matches = [[ value['type'], key] for key, value in quick_search.items() if query in str(value['template_string'])]
df = pd.DataFrame(matches, columns=['name', 'type'])
df


### define functions

In [ ]:
def list_to_dict(lst: list):
    _dict = {}
    for index, item in enumerate(lst[::2]):
        _i = index * 2
        _value = lst[_i+1];
        if type(_value) == list:
            _value = list_to_dict(_value[1])
        _dict[item] = _value
    return _dict

In [ ]:
def get_dependency_snippets(s: str):
    code_pattern = r"\{\{([^{}]*)\}\}"
    return re.findall(code_pattern, s)

global g_trigger_regex
g_trigger_regex = r"[a-zA-Z_0-9]+\.trigger"
def get_trigger_snippets(s: str):
    return re.findall(g_trigger_regex, s)

In [ ]:
def get_related_components(component):
    events = []
    dependencies = []
    triggers = []
    try:
        events_list = [lst[1] for lst in component["template"]["events"][1]]
        for event in events_list:
            events.append(list_to_dict(event))
    except:
        pass

    try:
        dependencies = get_dependency_snippets(str(component["template"]))
    except:
        pass

    try:
        triggers = get_trigger_snippets(str(component["template"]))
    except:
        pass

    return {
        "dependency_code_snippets": dependencies,
        "events": events,
        "trigger_code_snippets": triggers
    }

### get dependencies for each component

In [ ]:
for key in data.keys():
    data[key]["component_relations"] = get_related_components(data[key])

In [ ]:
with open("debug_files/debug_raw_component_relations.json", "w") as f:
    json.dump({k: v.get('component_relations') for k,v in data.items()}, f, indent=2 * " ")

### map components to the dependencies of other components

In [ ]:
def filter_triggers_from_code_snippets(snippets: list[str]):
    # replace all instances of regex in string with empty string
    return [re.sub(g_trigger_regex, "", snippet) for snippet in snippets]

In [ ]:
# test filter_triggers_from_code_snippets
test = ["test.trigger('test')", "test2.trigger('test_2')", "test3.trigger('test_3') test3.trigger('test_3') test4.trigger('test_4')"]
result = filter_triggers_from_code_snippets(test)
print(result)
assert result == ["('test')", "('test_2')", "('test_3') ('test_3') ('test_4')"]

In [ ]:
code_snippets = {
    key: " | ".join(filter_triggers_from_code_snippets(value["component_relations"]["dependency_code_snippets"]))
    for key, value 
    in data.items() 
    if len(value["component_relations"]["dependency_code_snippets"]) > 0
}

triggers = {
    key: {
        trigger_key: {
            "event": "code_trigger",
            "type": data[trigger_key].get("type"),
            "method": "trigger",
            "pluginId": trigger_key
        }
        for trigger_key
        in set([
            re.sub(".trigger", "", trigger_string)
            for trigger_string
            in value["component_relations"]["trigger_code_snippets"]
        ])
    }
    for key, value 
    in data.items() 
    if len(value["component_relations"]["trigger_code_snippets"]) > 0
}

events = {
    key: {
        event["pluginId"]: event
        for event 
        in value["component_relations"]["events"]
    }
    for key, value 
    in data.items() 
    if len(value["component_relations"]["events"]) > 0
}

##### split events into triggers and non-triggers

In [ ]:
non_trigger_events = {}

for key, value in events.items():
    for event_key, event_value in value.items():
        if event_value["method"] == "trigger":
            if not key in triggers:
                triggers[key] = {}
            triggers[key][event_key] = event_value
        elif event_value["pluginId"] != "":
            if not key in non_trigger_events:
                non_trigger_events[key] = {}
            non_trigger_events[key][event_key] = event_value

In [ ]:
with open("debug_files/debug_triggers.json", 'w') as f:
    json.dump(triggers, f, indent=2 * " ")

##### get dependency list -- search code_snippets for component-name + '.'

In [ ]:
dependencies = {key: [] for key in code_snippets.keys()}
for parent, snippets in code_snippets.items():
    for dependent in code_snippets.keys():
        if f"{dependent}." in snippets:
            dependencies[parent].append(dependent)

for key, value in dependencies.items():
    data[key]["component_relations"]["dependencies"] = value

### precompute reverse relations : 
#### grab a list of all components that depend on a given component
- e.g this component updates this list of components

In [ ]:
# reverse dependencies map into dependants map
for parent in dependencies.keys():
    if not "component_relations" in data[parent]:
        data[parent]["component_relations"] = {}
    if not "dependants" in data[parent]["component_relations"]:
        data[parent]["component_relations"]["dependants"] = []
    for dependant, dependency_components in dependencies.items():
        # if parent is a dependency of dependant
        if parent in dependency_components:
            data[parent]["component_relations"]["dependants"].append({
                "event": "on_value_change",
                "type": data[dependant].get("type"),
                "method": "update",
                "pluginId": dependant,
                "triggered_by": parent,
                "derived_event": True
            })

In [ ]:
def add_events_to_dependants_list(events_map: dict):
    global data;
    for parent, events in events_map.items():
        if not "component_relations" in data[parent]:
            data[parent]["component_relations"] = {}
        if not "dependants" in data[parent]["component_relations"]:
            data[parent]["component_relations"]["dependants"] = []
        for event in events.values():
            event["triggered_by"] = parent
            data[parent]["component_relations"]["dependants"].append(event)

In [ ]:
add_events_to_dependants_list(triggers)
add_events_to_dependants_list(non_trigger_events)

In [ ]:
with open("debug_files/debug_dependencies.json", "w") as f:
    json.dump({k: v.get('component_relations') for k,v in data.items()}, f, indent=2 * " ")

#### intermediate output files

In [ ]:
dependency_graph = {key: value["component_relations"] for key, value in data.items()}
with open("intermediate_outputs/dependency_graph.json", "w") as f:
    json.dump(dependency_graph, f, indent=2 * " ")

In [ ]:
with open("intermediate_outputs/formatted_appState_w_dependencies.json", 'w') as f:
    f.write(json.dumps(data, indent=2 * " "))